<a href="https://colab.research.google.com/github/FranciaAlday/Proyectos/blob/main/PPCienciaDatos/Clases/6Sep_hash_prepro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Poner siempre documentación
*   No imprimir en funciones, mejor devolver el resultado

# Funciones de hash

Bit twiddling hacks by Sean Eron A.

In [ ]:
def division(x=0, M=887):
  return abs(x)%M

print(division(-30502))
print(division(30502))
print(division(30501))

for i in range(10):
  print(i, ' = ', division(i))

344
344
343
0  =  0
1  =  1
2  =  2
3  =  3
4  =  4
5  =  5
6  =  6
7  =  7
8  =  8
9  =  9


In [ ]:
# Obtenemos la sección central de los bits, los k bits centrales
def mitad_cuadrado(x=0, k=10, w=32):
  mask = (1 << k) - 1 #mascara <<corrimiento hacia la izquierda 10000 - 1 -> 01111
  return ((x**2) >> (w-k)) & mask # >>corrimiento hacia la derecha (w-k) bits &(a nivel de bits) mask(01111)

print(mitad_cuadrado(-30502))
print(mitad_cuadrado(30502))
print(mitad_cuadrado(30501))

for i in range(10):
  print(i, ' = ', mitad_cuadrado(i)) # da puros ceros porque son numeros muy pequeños, hace que el último numero (2 o 1) se vaya y de puros ceros

221
221
221
0  =  0
1  =  0
2  =  0
3  =  0
4  =  0
5  =  0
6  =  0
7  =  0
8  =  0
9  =  0


## Método Leonardo de Pisa: Fibonacci para 32 bits a=2654435769

In [ ]:
# Este se utiliza en muchos lenguajes de programación
def fibonacci(x=0, k=10, w=32, a=2654435769):
  mask = (1 << k) - 1 #mascara <<corrimiento hacia la izquierda 10000 - 1 -> 01111
  return ((x*a) >> (w-k)) & mask # >>corrimiento hacia la derecha (w-k) bits &(a nivel de bits) mask(01111)
# aqui la semilla es x

print(fibonacci(-30502))
print(fibonacci(30502))
print(fibonacci(30501))

for i in range(10):
  print(i, ' = ', fibonacci(i))

# también sirve para generar números pseudo aleatorios con este método de hasheo fibonacci

744
279
670
0  =  0
1  =  632
2  =  241
3  =  874
4  =  483
5  =  92
6  =  725
7  =  334
8  =  966
9  =  575


Para cualquier serie de Fibonacci de 10 digitos su suma es el séptimo dígito por 11

# Preprocesamiento

## Datos faltantes

In [ ]:
import pandas as pd
import numpy as np
from io import StringIO #entrada-salida

In [ ]:
datos = \
"""A,B,C,D
1.1,2.4,3.6,4.2
5.6,6.9,8.1
9.1,0.6,1.2,"""
df = pd.read_csv(StringIO(datos))
df

,A,B,C,D
0,1.1,2.4,3.6,4.2
1,5.6,6.9,8.1,NaN
2,9.1,0.6,1.2,NaN


In [ ]:
# Para saber si hay valores nulos esta el método isnull
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,False,False,True
2,False,False,False,True


In [ ]:
# Para saber cuantos nulos hay por columna
df.isnull().sum()

A    0
B    0
C    0
D    2
dtype: int64

In [ ]:
# Para deshacerse de los nulos por columna(axis=1), por fila(axis=0)
df.dropna(axis=1)

,A,B,C
0,1.1,2.4,3.6
1,5.6,6.9,8.1
2,9.1,0.6,1.2


In [ ]:
df.dropna(how='all')

,A,B,C,D
0,1.1,2.4,3.6,4.2
1,5.6,6.9,8.1,NaN
2,9.1,0.6,1.2,NaN


In [ ]:
df.dropna(thresh=4)

,A,B,C,D
0,1.1,2.4,3.6,4.2


In [ ]:
df.dropna(subset=['A','D'])

,A,B,C,D
0,1.1,2.4,3.6,4.2


Si eliminamos por filas o columnas nos quedamos sin la mayoría de los datos, entonces mejor asignamos valores

## Asignar valores

In [ ]:
from re import M
#Sustituir valores nulos
# hacemos una copia
datos_ = df.copy()
for col in df.columns.values:
  falta = np.sum(df[col].isnull())
  if falta:
    print('Asignado {} valores en columna {}'.format(falta,col))
    mean = df[col].mean()
    datos_[col] = df[col].fillna(mean)
datos_
# Les asigna el mean por columna

Asignado 2 valores en columna D


,A,B,C,D
0,1.1,2.4,3.6,4.2
1,5.6,6.9,8.1,4.2
2,9.1,0.6,1.2,4.2


In [ ]:
# Para sustituir valores nulos por fila


## Sustituir valores nulos con sklearn

No permite hacerlo por filas, sólo por columnas. Una versión anterior si lo podía hacer.

Por lo general en un dataframe la media de las variables se saca por columnas, entonces no tiene sentido sacar media por filas si no tiene que ver con esa variable.

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(df.values)
datos_ = imp.transform(df.values)
datos_

array([[1.1, 2.4, 3.6, 4.2],
       [5.6, 6.9, 8.1, 4.2],
       [9.1, 0.6, 1.2, 4.2]])

In [ ]:
df = pd.DataFrame([['a','x'],['a',np.nan],['b','y'],['c','x'],[np.nan,'x']], dtype='category')
imp = SimpleImputer(strategy='most_frequent')
print(imp.fit_transform(df))

[['a' 'x']
 ['a' 'x']
 ['b' 'y']
 ['c' 'x']
 ['a' 'x']]


In [ ]:
# Si hay misma cantidad de elementos rellena por orden
df = pd.DataFrame([['a','y'],['a',np.nan],['b','y'],['b','x'],[np.nan,'x']], dtype='category')
imp = SimpleImputer(strategy='most_frequent')
print(imp.fit_transform(df))

[['a' 'y']
 ['a' 'x']
 ['b' 'y']
 ['b' 'x']
 ['a' 'x']]


In [ ]:
df = pd.DataFrame([['verde','M', 10.3, 'clase1'],
                   ['azul','G', 15.7, 'clase2'],
                   ['rojo','XG', 10.0, 'clase2'],
                   ['aqua','M', 39.4, 'clase3'],])
df.colums = ['color','talla','precio','clase']
df

<ipython-input-28-0fa8aa57766a>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.colums = ['color','talla','precio','clase']


,0,1,2,3
0,verde,M,10.3,clase1
1,azul,G,15.7,clase2
2,rojo,XG,10.0,clase2
3,aqua,M,39.4,clase3


In [ ]:
talla_map = {'XG':3, 'G':2, 'M':1}
df['talla'] = df['talla'].map(talla_map)
df

KeyError: ignored

In [ ]:
import numpy as np
# Con un diccionario por entendimiento:
clase_map = {et:ind for ind,et in enumerate(np.unique(df.clase))}
clase_map

In [ ]:
df['clase'] = df['clase'].map(clase_map)
df

## One-hot encoder

## Dummies

In [ ]:
pd.get_dummies(df)

,2,0_aqua,0_azul,0_rojo,0_verde,1_G,1_M,1_XG,3_clase1,3_clase2,3_clase3
0,10.3,0,0,0,1,0,1,0,1,0,0
1,15.7,0,1,0,0,1,0,0,0,1,0
2,10.0,0,0,1,0,0,0,1,0,1,0
3,39.4,1,0,0,0,0,1,0,0,0,1


## Normalización

In [ ]:
ejem = np.array([0,1,2,3,4,5])
print('Normalizando : ',(ejem-ejem.min()/(ejem.max()-ejem.min())))

Normalizando :  [0. 1. 2. 3. 4. 5.]


## Estandarización

Leva los valores a media .. y desviación estándar

In [ ]:
ejem = np.array([0,1,2,3,4,5])
print('Estandarizado : ',((ejem-ejem.mean())/ejem.std()))

Estandarizado :  [-1.46385011 -0.87831007 -0.29277002  0.29277002  0.87831007  1.46385011]
